In [1]:
from numba import jit
from numpy import arange

# jit decorator tells Numba to compile this function.
# The argument types will be inferred by Numba when function is called.
@jit
def sum2d(arr):
    M, N = arr.shape
    result = 0.0
    for i in range(M):
        for j in range(N):
            result += arr[i,j]
    return result

a = arange(9).reshape(3,3)
print(sum2d(a))

36.0


In [10]:
def sum2dP(arr):
    M, N = arr.shape
    result = 0.0
    for i in range(M):
        for j in range(N):
            result += arr[i,j]
    return result

In [14]:
print(sum2dP(a))

36.0


In [20]:
import timeit
print(timeit.timeit(lambda:sum2d(a), number = 1000000))
print(timeit.timeit(lambda:sum2dP(a), number = 1000000))

0.31943458691239357
4.211598303867504


In [21]:
4.2/0.32


13.125